In [1]:
import sys
sys.path.append('/home/jra21/work/source/repos/opep')

In [2]:
from openep import io as openep_io
from openep import case as openep_case
from openep import mesh_routines as openep_mesh
from openep import case_routines as case_routines
from openep import draw_routines as draw
import numpy as np

In [3]:
filename = '/home/jra21/work/source/repos/opep/examples/data/new_dataset_2.mat'
distance_thresh = 10

ep_case = openep_io.load_case(filename)

In [ ]:
# Anatomic descriptions (Mesh) - nodes and indices
pts = ep_case.nodes
indices = ep_case.indices


In [ ]:
# Electric data
# Locations – Cartesian co-ordinates, projected on to the surface 
locations = case_routines.get_electrogram_coordinates(ep_case,'type','bip')
i_egm = ep_case.electric['egm'].T
i_vp = case_routines.getMappingPointsWithinWoI(ep_case)
# macthing the shape of ivp with data
i_vp_egm = np.repeat(i_vp, repeats=i_egm.shape[1], axis=1)
# macthing the shape of ivp with coords
i_vp_locations = np.repeat(i_vp, repeats=locations.shape[1],axis=1)

# Replacing the values outside the window of interest with Nan values
i_egm[~i_vp_egm] = np.nan
locations[~i_vp_locations] = np.nan

# For each mapping point, n, find the voltage amplitude
max_volt = np.amax(a=i_egm,axis=1).reshape(len(i_egm),1)
min_volt = np.amin(a=i_egm,axis=1).reshape(len(i_egm),1)

amplitude_volt = np.subtract(max_volt,min_volt)

for indx in range(amplitude_volt.shape[1]):
    temp_data = amplitude_volt[:,indx]
    temp_coords = locations
    i_nan = np.isnan(temp_data)
    temp_data=temp_data[~i_nan]
    temp_coords=temp_coords[~i_nan]


    interp = case_routines.OpenEPDataInterpolator(method='rbf',distanceThreshold=distance_thresh,rbfConstant=1)
    vertex_voltage_data = interp.interpolate(x0=temp_coords,d0=temp_data,x1=pts)



vsurf = draw.DrawMap(ep_case,
        volt = vertex_voltage_data,
        freeboundary_color='black',
        cmap='jet_r',
        freeboundary_width=5,
        minval=0,
        maxval=2,
        volt_below_color='brown', 
        volt_above_color='magenta', 
        nan_color='gray', 
        plot=True,)

In [ ]:
# import numpy as np

# def get_electrogram_coordinates(mesh_case, *args):
#     '''
    
#     '''
#     n_standard_args = 1
#     v_type = "bip"
    
#     nargin = len(args)+1
#     if nargin>n_standard_args:
#         for i in range(0,(nargin-n_standard_args),2):
#             if np.char.lower(args[i]) == "type":
#                 v_type = args[i+1]
    
#     if np.char.lower(v_type) == "bip":
#         x = mesh_case.electric["egmX"].T
        
#     elif np.char.lower(v_type) == "uni":
#         if not("egmUniX" in mesh_case.electric):
#             raise ValueError("There is no unipolar data associated with this openep case")
            
#         else:
#             x = mesh_case.electric['egmUniX'].T
        
#     return x

In [ ]:
# Load EGM voltage values
# UseCase 28 - Get the electrogam at a mapping point #39

def get_egms_at_points(mesh_case,):
    pass

In [ ]:
import numpy as np
hsurf1 = draw.getAnatomicalStructures(ep_case,plot=True)
py_length = np.array(hsurf1['Lengths'])
py_area = np.array(hsurf1['Area'])


mat_area = np.array([154.773515517340,260.175174080897,331.548738619316,508.782228060733,282.030461949226,949.773225470272,634.814247737892])
mat_length = np.array([48.4671072321950,66.1017537507639,67.9940320911940,82.9700863606623,63.4461422403505,120.374453837180,97.0835005157046])
for i in range(len(mat_length)):
    mat_length[i]=round(mat_length[i],4)

for j in range(len(mat_area)):
    mat_area[i]=round(mat_area[i],4)

diff_area = np.subtract(py_area,mat_area)`
print('difference-area:\n',diff_area)
diff_len = np.subtract(py_length,mat_length)
print('difference-length:\n',diff_len)

In [ ]:
import numpy as np

fb = [[14336, 14327],
       [14327, 14357],
       [14357, 14320]]
fb = np.array(fb).astype(np.int64)
print(fb[:,1])
print(fb[:,1][-1])

test_set = np.array(list(np.zeros((fb.shape[0],fb.shape[1]))))
test_set[:] = np.nan

test_set[:,0] = fb[:,0]
print(test_set)

# testset = [fb[:,1][-1], lambda x: for item in fb[1:len(fb-1)] ]


res = list(map(lambda x:x, fb[:,1][0:(len(fb)-1)]))
res.insert(0,fb[:,1][-1])
res = np.array(res)
print(res)

test_set[:,1] = res
print('\n')
print(test_set)

i_start = np.where(np.diff(a=test_set,n=1,axis=1).astype(np.int64))
print(i_start)

In [5]:
import trimesh as tm

pts = ep_case.nodes
face = ep_case.indices.astype(int)

ff = []
l = []

tri_mesh = tm.Trimesh(vertices=pts,
                        faces=face, 
                        process=False)

free_b = draw.free_boundary(tri_mesh)
print(free_b['connected_freeboundary'])
print(free_b['length'])

# freeboundary_vertex_nodes = []
# freeboundaries = []

# tri_outline = tri_mesh.outline().to_dict()
# mesh_outline_entities = tri_mesh.outline().entities
# no_of_freeboundaries = len(mesh_outline_entities)

#  # Find all the freeboundary facets
# for i in range(no_of_freeboundaries):
#     freeboundary_vertex_nodes.append(tri_outline["entities"][i]['points'])

# print(np.array(tri_outline['entities'][0]['points']).shape)
    
# # match the order of the freeboundaries, same as matlab
# for item in freeboundary_vertex_nodes:
#     print(item)
    


# f_fall = draw.get_freeboundaries(tri_mesh)
# fb = f_fall['freeboundary']

# for item in fb:
#     print(item)
# test_set=np.array(list(np.zeros((fb.shape[0],fb.shape[1]))))
# test_set[:] = np.nan
# test_set[:,0] = fb[:,0]

# #exctracting the contents of 2nd columm fb array into the testset
# second_col_fb = list(map(lambda x:x, fb[:,1][0:len(fb)-1]))
# second_col_fb.insert(0,fb[:,1][-1])
# second_col_fb = np.array(second_col_fb)

# # Assigning the values of the second column to the 2nd column of testset
# test_set[:,1] = second_col_fb


# print('test-set\n',test_set[0:69])
# # isstat
# # i_start = []
# i_start = np.where(np.diff(a=test_set,n=1,axis=1).astype(np.int64))[0]
# print('i-start\n',i_start)
# # print(np.diff(i_start))

# # mat_start = np.array([1,87,230,299,461,571,668])
# # print(np.diff(mat_start))



# if not list(i_start):    
#     ff.append(np.array(fb))
#     coords = draw.get_freeboundary_points(tri_mesh,ff[0])
#     l.append(draw.lineLength(coords))
    
# else:
#     for i in range(i_start.shape[0]):
#         if i<(i_start.shape[0]-1):
#             np.array(ff.append(fb[i_start[i]:i_start[i+1]]))
#             print('ff-shape\n',ff[i].shape)
#             print('ff\n',ff[i])
#             coords = draw.get_freeboundary_points(tri_mesh,ff[i])
#             print('coords\n',coords)
#             l.append(draw.lineLength(coords))
#             print('l\n',l)
#         else:
#             ff.append(fb[i_start[i]:])
#             print('ff\n',ff[i].shape)

#             coords = draw.get_freeboundary_points(tri_mesh,ff[i])
#             print('coords\n',coords)

#             l.append(draw.lineLength(coords))
#             print('l\n',l)

        
# print('ff\n',coords)
# print('lineLength',l)

# print(pts[FF['freeboundary'][:,1][0:69]])
# print(FF['freeboundary'][:,0])

# coords = pts[FF['freeboundary'][:,0][0:69]]
# x = coords[:,0]
# y = coords[:,1]
# z = coords[:,2]

# print('x\n',x.shape)
# print('y\n',y.shape)
# print('z\n',z.shape)

# coords = np.concatenate(x,y,z)
# print('coords\n',coords)

# f_f = FF['freeboundary'][:,0]
# fb_coords = draw.get_freeboundary_points(tri_mesh,FF['freeboundary'][0:69])
# print(fb_coords)


[array([[14336, 14327],
       [14327, 14357],
       [14357, 14320],
       [14320, 14325],
       [14325, 14339],
       [14339, 14319],
       [14319, 14326],
       [14326, 14335],
       [14335, 14322],
       [14322, 14355],
       [14355, 14354],
       [14354, 14351],
       [14351, 14334],
       [14334, 14350],
       [14350,  3618],
       [ 3618, 14333],
       [14333, 14349],
       [14349, 14321],
       [14321, 14331],
       [14331, 14369],
       [14369, 14323],
       [14323, 14344],
       [14344, 14367],
       [14367, 14332],
       [14332, 14346],
       [14346, 14345],
       [14345, 14366],
       [14366, 14375],
       [14375, 14348],
       [14348, 14374],
       [14374, 14373],
       [14373, 14380],
       [14380, 14364],
       [14364, 14372],
       [14372, 14363],
       [14363,   916],
       [  916, 14342],
       [14342, 14353],
       [14353, 14362],
       [14362, 14347],
       [14347, 14341],
       [14341, 14356],
       [14356, 14343],
       [14

In [ ]:
a = []

if not a:
    print("List is empty")
    
else:
    print('list not empty')